## **DATA SCIENCE CHALLENGE - MERCADO LIBRE**

Autor= Lorena Aldasalo

# **1.Instalamos los requerimientos.**

In [1]:
!pip install plotly==5.1.0
!pip install plotly_express==0.4.0 

# **2. Importamos las librerías de Python a utilizar**

In [2]:
import requests as rq
import pandas as pd
import numpy as np
import plotly.express as px

# **3. Descargamos los datos de la API de mercado libre**

Nota= Recibí ayuda en éste punto, dado que no tengo experiencia previa en el manejo de API consumidas por red. Mi mayor experiencia es en el consumo de datos en bases de datos, utilizando SQL (Oracle, SQL Server, Teradata, etc).

In [3]:
def busqueda(text, type_search="q", n=50):
    result = pd.DataFrame()
    for i in range(n//50+1):
        params = {type_search:text, "offset":i*50, "limit":min(50, n-i*50)}
        response = rq.get("https://api.mercadolibre.com/sites/MLA/search", params=params)
        if len(response.json()["results"]) > 0:
            tmp = pd.json_normalize(response.json()["results"])
            tmp["query"] = text
            result = result.append(tmp)
        else:
            break
    return result

In [4]:
def busqueda_multiple(query_list, n=50):
    result = pd.DataFrame()
    for query in query_list:
        result = result.append(busqueda(query, n=n))
    return result

In [5]:
# Toma las categorías de Argentina y su primera subcategoría 
def categorias():
    result = pd.DataFrame()
    resp1 = rq.get("https://api.mercadolibre.com/sites/MLA/categories")
    for subcat1 in resp1.json():
        resp2 = rq.get("https://api.mercadolibre.com/categories/" + subcat1["id"])
        tmp = pd.json_normalize(resp2.json()["children_categories"])
        tmp["categoria"] = subcat1["name"]
        result = result.append(tmp)
    return result

# **4. Análisis de publicaciones por categoría**
Observamos cuales son las categorías y subcategorías con mayor cantidad de publicaciones

In [6]:
cat = categorias()

In [7]:
cat.sort_values(by='total_items_in_this_category', ascending=False)

,id,name,total_items_in_this_category,categoria
13,MLA1747,Repuestos Autos y Camionetas,8501552,Accesorios para Vehículos
2,MLA412445,Libros,7605141,"Libros, Revistas y Comics"
1,MLA1174,Música,1181277,"Música, Películas y Series"
2,MLA1592,Bazar y Cocina,633981,"Hogar, Muebles y Jardín"
15,MLA1771,Repuestos Motos y Cuatriciclos,617424,Accesorios para Vehículos
...,...,...,...,...
2,MLA10434,Eventos Deportivos,37,Entradas para Eventos
0,MLA455033,Ácido Clorhídrico,18,Otras categorías
1,MLA454714,Eventos a Beneficio,8,Entradas para Eventos
7,MLA432903,Hornos Crematorios,6,Otras categorías


A continuación se puede observar gráficamente cuales son las 10 categorías con mayor cantidad de items con un sencillo gráfico de barras

In [8]:
cat_grafico = (cat.sort_values(by='total_items_in_this_category', ascending=False)).head(n=10)
fig = px.bar(cat_grafico, x="name", y="total_items_in_this_category")
fig.show()

Ahora no queremos ver sólo las 10 categorías con más item, sino queremos ver un mapa que nos ilustre todo. Para eso usamos un **Treemap**


In [9]:
fig = px.treemap(cat, path=[px.Constant("Todo"), 'categoria','name'], values='total_items_in_this_category')
fig.show()

# **5. Analisis de búsqueda "tv 4k"**
Buscamos la palabra "tv 4k" y nos traemos 1000 registros. Nos bajamos la tabla a excel para visualizar el dataset y en python calculamos algunas métricas.

In [10]:
#Selecciono 1000 Televisores de la lista, calculo el descuento y cual sería el descuento promedio
bm = busqueda("tv 4k",n=1000)

#Exportamos la tabla completa de 1000 registros a excel
bm.to_excel("bm.xlsx")

In [11]:
#Calculamos cual es el descuento para cada uno de los productos
descuento = (bm.original_price - bm.price)/bm.original_price

#Principales métricas de la variable "Descuento"

print("Cantidad = ",descuento.mean(skipna=True)) # En promedio hay un 16% de descuento)
print ("Cantidad con descuento = ", descuento.count()) #hay 103 televisores con descuento#
print ("Porcentaje de pub con descuento = ", (descuento.count()/len(bm))) #Porcentaje de casos con descuento 10,37%
print ("Mínimo descuento = ", descuento.min(skipna=True))
print ("Máximo descuento = ", descuento.max(skipna=True))

Cantidad =  0.16420522450010916
Cantidad con descuento =  102
Porcentaje de pub con descuento =  0.1027190332326284
Mínimo descuento =  0.010638524223919657
Máximo descuento =  0.35011890099845855


# 5.1 Exploración de variables numéricas de la búsqueda "tv 4k" 
A continuación se construyen las principales medidas estadísticas de las variables numéricas

In [12]:
#cuento cantidad de registros de variables numéricas#
#Podremos observar si existen valores nulos en cada una de las variables, ya que "count()" en Python omite valores nulos.
bm[["original_price","price","available_quantity","sold_quantity","seller.seller_reputation.transactions.total","seller.seller_reputation.transactions.canceled",
    "seller.seller_reputation.transactions.ratings.negative","seller.seller_reputation.transactions.ratings.positive",
    "seller.seller_reputation.transactions.ratings.neutral","seller.seller_reputation.transactions.completed","seller.seller_reputation.metrics.claims.rate",
    "seller.seller_reputation.metrics.claims.value","seller.seller_reputation.metrics.delayed_handling_time.rate",
    "seller.seller_reputation.metrics.delayed_handling_time.value","seller.seller_reputation.metrics.sales.completed",
    "seller.seller_reputation.metrics.cancellations.rate","installments.quantity","installments.amount","installments.rate"]].count()

original_price                                                  102
price                                                           993
available_quantity                                              993
sold_quantity                                                   993
seller.seller_reputation.transactions.total                     993
seller.seller_reputation.transactions.canceled                  993
seller.seller_reputation.transactions.ratings.negative          993
seller.seller_reputation.transactions.ratings.positive          993
seller.seller_reputation.transactions.ratings.neutral           993
seller.seller_reputation.transactions.completed                 993
seller.seller_reputation.metrics.claims.rate                    993
seller.seller_reputation.metrics.claims.value                   993
seller.seller_reputation.metrics.delayed_handling_time.rate     993
seller.seller_reputation.metrics.delayed_handling_time.value    993
seller.seller_reputation.metrics.sales.completed

Conclusión = Se observan 4 variables con valores nulos:


*   original_price
*   installments.quantity
*   installments.amount
*   installments.rate





In [13]:
#Cálculo de medidas estadísticas básicas
bm.agg({'price': ['mean','median','min', 'max'], 'available_quantity': ['mean','median','min', 'max'],'sold_quantity':['mean','median','min', 'max'], 
        'seller.seller_reputation.transactions.total':['mean','median','min', 'max'],
        'seller.seller_reputation.transactions.canceled': ['mean','median','min', 'max'],
        'seller.seller_reputation.transactions.ratings.negative': ['mean','median','min', 'max'],
        'seller.seller_reputation.transactions.ratings.positive': ['mean','median','min', 'max'],
        'seller.seller_reputation.transactions.ratings.neutral':['mean','median','min', 'max'],
        'seller.seller_reputation.transactions.completed': ['mean','median','min', 'max'],
        'seller.seller_reputation.metrics.claims.rate': ['mean','median','min', 'max'],
        'seller.seller_reputation.metrics.claims.value': ['mean','median','min', 'max'],
        'seller.seller_reputation.metrics.delayed_handling_time.rate': ['mean','median','min', 'max'],
        'seller.seller_reputation.metrics.delayed_handling_time.value':['mean','median','min', 'max'],
        'seller.seller_reputation.metrics.sales.completed': ['mean','median','min', 'max'],
        'seller.seller_reputation.metrics.cancellations.rate':['mean','median','min', 'max'],
        'installments.quantity': ['mean','median','min', 'max'],
        'installments.amount':['mean','median','min', 'max'],
        'installments.rate':['mean','median','min', 'max']})

,price,available_quantity,sold_quantity,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,installments.quantity,installments.amount,installments.rate
mean,6.938576e+04,225.467271,41.566969,40499.23565,2173.941591,0.02138,0.960443,0.018177,38325.294058,0.009929,102.485398,0.022123,112.705942,7804.040282,0.003099,11.460606,8796.177303,53.159091
median,6.399900e+04,1.000000,2.000000,15415.00000,906.000000,0.02000,0.970000,0.010000,13948.000000,0.011200,28.000000,0.007800,15.000000,2848.000000,0.001000,12.000000,7749.920000,70.170000
min,1.000000e+02,1.000000,0.000000,3.00000,0.000000,0.00000,0.850000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,6.000000,14.180000,0.000000
max,1.040000e+06,50000.000000,6460.000000,547217.00000,25444.000000,0.09000,1.000000,0.130000,521773.000000,0.039700,1385.000000,0.177500,2102.000000,130794.000000,0.049100,12.000000,99124.030000,70.170000


Conclusión= La comparación entre el promedio y mediana para cada una de las variables numéricas nos permite detectar la presencia de valores outliers o cierta concentración en un determinado valor.
Las variables que difieren significativamente son las siguientes:

*   Variable "available_quantity": tiene el valor de 1 como mediana y 225 como media debido a que hay muchos vendedores que tienen 1 unidad disponible del producto
*   Variable "sold quantity": Hay vendedores que vendieron pocas unidades, por esa razón la mediana es de 2 y la media es más alta.






Llama la atención en el anterior análisis que el precio mínimo de un televisos sea de $100, por esa razón se sospecha que haya quizás productos usados que presenten valores inferiores. Por ese motivo se decidió analizar la variable "condition" con el precio.

In [14]:
bm.groupby('condition').agg({'price': ['mean','min', 'max']})

price                   
                   mean     min        max
condition                                 
new        71690.419464  199.91  1040000.0
used       34173.647049  100.00   317000.0

Conclusión: Se observa que si bien el promedio de precio de los artículos usados es inferior a los nuevos, el precio mínimo de los articulos nuevos es de $199,91.
Buscamos ese producto en particular y observamos que se trata de un cable para conectar al tv, por ese motivo tiene un precio inferior.
Esto nos dá la pauta que necesitamos hacer una mejor selección o limpieza del dataset a la hora de construir un modelo.

# 5.2 Rank de variables numéricas

Otro de los tratamientos que se realizan muy a menudo al hacer análisis exploratorio y contruir modelos predictivos, es el armado de "ranks" de las variables numéricas.
 
Para ello se ordenan los valores de menor a mayor y se contruyen grupos con la misma cantidad de casos.
De esta forma se puede observar como se comporta cada grupo del rank con respecto a la variable objetivo.

Para éste ejercio se armaron "quintiles", es decir 5 grupos del mismo tamaño.

In [15]:
#Se crea la variable "descuento" y "Porcentaje de descuento" y se agregan al dataset
bm['descuento']=bm.original_price-bm.price
bm['porc_descuento']=(bm.original_price-bm.price)/bm.original_price

In [16]:
#Se crea la variable "rank_xxx" de las variables creadas anteriormente
bm['rank_descuento'] = pd.qcut(bm['descuento'], 5, labels = False)
bm['rank_porc_descuento'] = pd.qcut(bm['porc_descuento'],5,labels=False)

In [17]:
#Se crean rank de algunas variables numéricas que componen dataset#
bm['rank_price']= pd.qcut(bm['price'], 5, labels = False)
bm['rank_seller.seller_reputation.t.total']= pd.qcut(bm['seller.seller_reputation.transactions.total'], 5, labels = False)
bm['rank_seller.seller_reputation.t.canceled']= pd.qcut(bm['seller.seller_reputation.transactions.canceled'], 5, labels = False)
bm['rank_seller.seller_reputation.t.ratings.positive']= pd.qcut(bm['seller.seller_reputation.transactions.ratings.positive'], 5, labels = False)
bm['rank_seller.seller_reputation.t.completed']= pd.qcut(bm['seller.seller_reputation.transactions.completed'], 5, labels = False)
bm['rank_seller.seller_reputation.metrics.claims.value']= pd.qcut(bm['seller.seller_reputation.metrics.claims.value'], 5, labels = False)
bm['rank_seller.seller_reputation.metrics.sales.completed']= pd.qcut(bm['seller.seller_reputation.metrics.sales.completed'], 5, labels = False)
bm['rank_installments.amount']= pd.qcut(bm['installments.amount'], 5, labels = False)

**Porcentaje de descuento**: Por cada rank se obtiene la cantidad, valor mínimo, valor máximo, y sumatoria de ventas

In [18]:
bm.groupby('rank_porc_descuento').agg({'porc_descuento': ['count','min', 'max'] , 'sold_quantity': 'sum'})

porc_descuento                     sold_quantity
                             count       min       max           sum
rank_porc_descuento                                                 
0.0                             97  0.010639  0.350119          4851
2.0                              2  0.130436  0.149684           101
4.0                              3  0.291143  0.348870           555

**Price**: Por cada rank se obtiene la cantidad, valor mínimo, valor máximo, y sumatoria de ventas

In [19]:
bm.groupby('rank_price').agg({'price' : ['count','min','max','mean'],'sold_quantity':'sum'})

price                                    sold_quantity
           count      min        max           mean           sum
rank_price                                                       
0            199    100.0     2995.0    1045.133015          8786
1            198   2999.0    56432.0   18669.382778         16721
2            205  56599.0    69999.0   63656.151220          6804
3            192  69999.9    90481.0   79382.242448          8047
4            199  90800.0  1040000.0  184445.440201           918

Conclusión: Se observa que cuando el precio es mayor a $2.999 (rank 1) las ventas decrecen cuando el precio del producto es mayor.

**seller.seller_reputation.transactions.total**: Por cada rank se obtiene la cantidad, valor mínimo, valor máximo, y sumatoria de ventas

In [20]:
bm.groupby('rank_seller.seller_reputation.t.total').agg({'seller.seller_reputation.transactions.total' : ['count','min','max','mean'],'sold_quantity':'sum'})

seller.seller_reputation.transactions.total  ... sold_quantity
                                                                            count  ...           sum
rank_seller.seller_reputation.t.total                                              ...              
0                                                                             201  ...          1350
1                                                                             196  ...          4057
2                                                                             203  ...          3250
3                                                                             197  ...          4367
4                                                                             196  ...         28252

[5 rows x 5 columns]

Conclusión: A partir de 56.126 transacciones totales (rank 4) se observa que la cantidad total de ventas aumentan significativamente. En los anteriores rank no se observa una relación directa con la cantidad de ventas

**seller.seller_reputation.transactions.canceled**: Por cada rank se obtiene la cantidad, valor mínimo, valor máximo, y sumatoria de ventas

In [21]:
bm.groupby('rank_seller.seller_reputation.t.canceled').agg({'seller.seller_reputation.transactions.canceled' : ['count','min','max','mean'],'sold_quantity':'sum'})

seller.seller_reputation.transactions.canceled  ... sold_quantity
                                                                                  count  ...           sum
rank_seller.seller_reputation.t.canceled                                                 ...              
0                                                                                   199  ...          1595
1                                                                                   198  ...          4535
2                                                                                   199  ...          2606
3                                                                                   207  ...          6411
4                                                                                   190  ...         26129

[5 rows x 5 columns]

Conclusión: Ocurre lo mismo que con la variable anterior.

**seller.seller_reputation.transactions.ratings.positive**: Por cada rank se obtiene la cantidad, valor mínimo, valor máximo, y sumatoria de ventas

In [22]:
bm.groupby('rank_seller.seller_reputation.t.ratings.positive').agg({'seller.seller_reputation.transactions.ratings.positive' : ['count','min','max','mean'],'sold_quantity':'sum'})

seller.seller_reputation.transactions.ratings.positive  ... sold_quantity
                                                                                                  count  ...           sum
rank_seller.seller_reputation.t.ratings.positive                                                         ...              
0                                                                                               263      ...         16233
1                                                                                               213      ...         10691
2                                                                                               179      ...          4230
3                                                                                               241      ...          8568
4                                                                                                97      ...          1554

[5 rows x 5 columns]

No hay una relación directa entre ratings posivitos y ventas

**seller.seller_reputation.transactions.completed**: Por cada rank se obtiene la cantidad, valor mínimo, valor máximo, y sumatoria de ventas



In [23]:
bm.groupby('rank_seller.seller_reputation.t.completed').agg({'seller.seller_reputation.transactions.completed' : ['count','min','max','mean'],'sold_quantity':'sum'})

seller.seller_reputation.transactions.completed  ... sold_quantity
                                                                                    count  ...           sum
rank_seller.seller_reputation.t.completed                                                  ...              
0                                                                                     201  ...          1350
1                                                                                     196  ...          4057
2                                                                                     203  ...          3250
3                                                                                     195  ...          4382
4                                                                                     198  ...         28237

[5 rows x 5 columns]

Se observa que cuando las transacciones son mayores a 55.250 (rank 4) aumentan significativamente las ventas

**seller.seller_reputation.metrics.claims.value**: Por cada rank se obtiene la cantidad, valor mínimo, valor máximo, y sumatoria de ventas

In [24]:
bm.groupby('rank_seller.seller_reputation.metrics.claims.value').agg({'seller.seller_reputation.metrics.claims.value' : ['count','min','max','mean'],'sold_quantity':'sum'})

seller.seller_reputation.metrics.claims.value  ... sold_quantity
                                                                                           count  ...           sum
rank_seller.seller_reputation.metrics.claims.value                                                ...              
0                                                                                            248  ...          2529
1                                                                                            156  ...          2342
2                                                                                            194  ...          4894
3                                                                                            200  ...          5146
4                                                                                            195  ...         26365

[5 rows x 5 columns]

**installments.amount**:
Por cada rank se obtiene la cantidad, valor mínimo, valor máximo, y sumatoria de ventas

In [25]:
bm.groupby('rank_installments.amount').agg({'installments.amount' : ['count','min','max','mean'],'sold_quantity':'sum'})

installments.amount  ... sold_quantity
                                       count  ...           sum
rank_installments.amount                      ...              
0.0                                      198  ...          8784
1.0                                      201  ...         20829
2.0                                      196  ...         10323
3.0                                      197  ...           899
4.0                                      198  ...           439

[5 rows x 5 columns]

Conclusión: Se observa que a partir del rank 1 decrece la cantidad de ventas en la medida que se incrementa la variable "installment. amount"

# 5.3. Análisis de variables categóricas

**Listing_type_id:** 
Para cada categoría se obtiene la cantidad de registros el promedio y la sumatoria de la variable "sold quantity"

In [26]:
bm.groupby('listing_type_id').agg({'sold_quantity':['count','mean','sum']})

sold_quantity                  
                        count       mean    sum
listing_type_id                                
gold_pro                  240  73.733333  17696
gold_special              753  31.314741  23580

Los "Gold pro" venden más en promedio que los "gold special"

**seller.eshop.nick_name:**
Armamos una tabla con los vendedores ordenando los que poseen más cantidad de ventas

In [27]:
vendor = bm.groupby('seller.eshop.nick_name').agg({'sold_quantity':['count','mean','sum']})
vendor.sort_values(by=('sold_quantity',  'sum'), ascending=False)

sold_quantity                   
                               count         mean   sum
seller.eshop.nick_name                                 
ELECTRO POINT                      3  1793.000000  5379
TIENDASHOPEALO                     3   535.000000  1605
BIDCOM                             5   152.000000   760
AYKIKESTORE                        4   188.750000   755
SEBARUMEL                         13    50.615385   658
...                              ...          ...   ...
ROSARIOALCOSTOSRL                  3     0.000000     0
ELECTROSERVICE3                   12     0.000000     0
DVTTIENDA                          1     0.000000     0
SHOWMUSIC                          1     0.000000     0
AIRSHOP1                           2     0.000000     0

[163 rows x 3 columns]

**address.state_name:** Analizamos cuales son las provincias donde hay más cantidad de ventas y luego construimos la variable "zona" en donde agrupamos a las provincias con pocos casos

In [28]:
zona = bm.groupby('address.state_name').agg({'sold_quantity':['count','mean','sum']})
zona.sort_values(by=('sold_quantity',  'sum'), ascending=False)

sold_quantity                  
                           count       mean    sum
address.state_name                                
Capital Federal              545  37.722936  20559
Buenos Aires                 316  62.107595  19626
Córdoba                       59   8.576271    506
Santa Fe                      47   7.787234    366
Tucumán                       17   9.941176    169
Entre Ríos                     3  16.666667     50
Mendoza                        2   0.000000      0
Neuquén                        3   0.000000      0
Santa Cruz                     1   0.000000      0

En la tabla anterior se observa que Capital Federal y Buenos Aires concentran la mayor cantidad de ventas

**shipping.mode**
Por cada categoría de la variable traemos la cantidad de registros, el promedio de ventas y la sumatoria de ventas

In [29]:
bm.groupby('shipping.mode').agg({'sold_quantity':['count','mean','sum']})

sold_quantity                  
                      count       mean    sum
shipping.mode                                
custom                   66  18.454545   1218
me1                      12   7.916667     95
me2                     749  52.251001  39136
not_specified           166   4.981928    827

En la anterior tabla se observa que "me2" de la variable "shipping.mode" tiene un promedio de ventas mayor

**shipping.logistic_type**
Por cada categoría de la variable traemos la cantidad de registros, el promedio de ventas y la sumatoria de ventas

In [30]:
bm.groupby('shipping.logistic_type').agg({'sold_quantity':['count','mean','sum']})

sold_quantity                   
                               count        mean    sum
shipping.logistic_type                                 
cross_docking                    379   51.005277  19331
custom                            66   18.454545   1218
default                           12    7.916667     95
drop_off                         196   10.127551   1985
fulfillment                       36  427.555556  15392
not_specified                    166    4.981928    827
xd_drop_off                      138   17.594203   2428

**domain_id**: Por cada categoría de la variable traemos la cantidad de registros, el promedio de ventas y la sumatoria de ventas

In [31]:
bm.groupby('domain_id').agg({'sold_quantity':['count','mean','sum']})

sold_quantity                   
                                                   count        mean    sum
domain_id                                                                  
MLA-AIR_CONDITIONER_REMOTE_CONTROLS                    2    5.000000     10
MLA-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS               90   31.433333   2829
MLA-AUDIO_AND_VIDEO_CONVERTERS                         1    1.000000      1
MLA-DATA_CABLES_AND_ADAPTERS                           2    1.000000      2
MLA-ELECTRONIC_ACCESSORIES_AND_SPARE_PARTS             6  102.500000    615
MLA-LED_CHIPS                                          1    5.000000      5
MLA-MINI_PCS                                           2   75.000000    150
MLA-SCREWS                                             6   23.833333    143
MLA-STREAMING_MEDIA_DEVICES                           74  156.175676  11557
MLA-STREAMING_MEDIA_PLAYER_REMOTE_CONTROLS             2    3.000000      6
MLA-TELEVISIONS                                      689   27.374456  18861
MLA-THERMOMETERS                                       1   50.000000     50
MLA-TRANSISTORS                                        1    5.000000      5
MLA-TV_AND_MONITOR_MOUNTS                             17  101.882353   1732
MLA-TV_RECEIVERS_AND_DECODERS                          5   37.000000    185
MLA-TV_REMOTE_CONTROLS                                92   55.380435   5095
MLA-TV_REMOTE_CONTROL_COVERS                           1   25.000000     25
MLA-UNIVERSAL_REMOTE_CONTROLS                          1    5.000000      5

En la anterior tabla se observa claramente que no todos corresponden a televisores, sino que también hay ventas de cables, controles remotos, etc. Con lo cual es necesaria una mayor limpieza del dataset para poder construir un modelo

In [32]:
#Construcción de variable con principales dominios de la búsqueda 
conditionlist = [(bm['domain_id'] == 'MLA-TELEVISIONS'), bm['domain_id'] == 'MLA-TV_REMOTE_CONTROLS', bm['domain_id'] == 'MLA-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS', bm['domain_id'] == 'MLA-STREAMING_MEDIA_DEVICES'] 
choicelist = ['1.MLA-TELEVISIONS','2.MLA-TV_REMOTE_CONTROLS','3.MLA-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS','4.MLA-STREAMING_MEDIA_DEVICES']
bm['Tipo_Dominio'] = np.select(conditionlist, choicelist, default='OTHERS')

In [33]:
bm.groupby('Tipo_Dominio').agg({'sold_quantity':['count','mean','sum']})

sold_quantity                   
                                                  count        mean    sum
Tipo_Dominio                                                              
1.MLA-TELEVISIONS                                   689   27.374456  18861
2.MLA-TV_REMOTE_CONTROLS                             92   55.380435   5095
3.MLA-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS            90   31.433333   2829
4.MLA-STREAMING_MEDIA_DEVICES                        74  156.175676  11557
OTHERS                                               48   61.125000   2934